# Access DB2 on Cloud using Python

In this notebook we established a connection to a database instance of DB2 Warehouse on Cloud from a Python notebook using ibm_db API. Then created a table and insert a few rows of data into it. Then queried the data. We also retrieved the data into a pandas dataframe.

## Import the ibm_db Python library

In [1]:
import ibm_db

## Identify the database connection credentials

 Connecting to dashDB or DB2 database requires the following information:

- Driver Name
- Database name
- Host DNS name or IP address
- Host port
- Connection protocol
- User ID (or username)
- User Password

In [2]:
dsn_hostname = 'dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net'
dsn_uid = 'zdd72371' 
dsn_pwd = '7xdr0-0zsnkzd76p' 
dsn_driver = '{IBM DB2 ODBC DRIVER}' 
dsn_database = 'BLUDB' 
dsn_port = '50000' 
dsn_protocol = 'TCPIP' 

## Create and Establish Connection

Lets build the dsn connection string using the credentials we entered above

In [3]:
dsn = ("DRIVER={};DATABASE={};HOSTNAME={};PORT={};PROTOCOL={};UID={};PWD={};"
      ).format(dsn_driver,dsn_database,dsn_hostname,dsn_port,dsn_protocol,dsn_uid,dsn_pwd)

print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=zdd72371;PWD=7xdr0-0zsnkzd76p;


Now establish the connection to the database

In [4]:
try:
    conn = ibm_db.connect(dsn,"","")
    print("Connected!")

except:
    print("Not connected!")

Connected!


## Create a table in the databse

In this step we will create a table in the database with following details:

<img src = "https://ibm.box.com/shared/static/ztd2cn4xkdoj5erlk4hhng39kbp63s1h.jpg" align="center">

In [5]:
drop_query = "drop table Instructor"
ibm_db.exec_immediate(conn,drop_query)

In [6]:
create_query = "create table Instructor(id int primary key not null, fname varchar(20), lname varchar(20), city varchar(20), ccode char(2))"
ibm_db.exec_immediate(conn,create_query)

## Insert data into the table

In this step we will insert some rows of data into the table.

The INSTRUCTOR table contains 3 rows of data:

<img src="https://ibm.box.com/shared/static/j5yjassxefrjknivfpekj7698dqe4d8i.jpg" align="center">

In [7]:
insert_query = "insert into Instructor(id,fname,lname,city,ccode) values (1,'Rav','Ahuja','TORONTO','CA'),\
                (2, 'Raul', 'Chong', 'Markham', 'CA'),\
                (3, 'Hima', 'Vasudevan', 'Chicago', 'US')"
ibm_db.exec_immediate(conn,insert_query)

## Query data in the table

In this step we will retrieve data we inserted into the INSTRUCTOR table.

In [8]:
select_query = "select * from Instructor"
select_stmt = ibm_db.exec_immediate(conn,select_query)

In [9]:
ibm_db.fetch_both(select_stmt)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'TORONTO',
 3: 'TORONTO',
 'CCODE': 'CA',
 4: 'CA'}

In [10]:
df= []
sql = "SELECT * FROM Instructor"
stmt = ibm_db.exec_immediate(conn, sql)
dictionary = ibm_db.fetch_assoc(stmt)
while dictionary != False:
    print(dictionary["ID"],dictionary["FNAME"],dictionary["LNAME"])
    df.append(dictionary)
    dictionary = ibm_db.fetch_assoc(stmt)
    

1 Rav Ahuja
2 Raul Chong
3 Hima Vasudevan


In [11]:
import pandas as pd
pd.DataFrame(df)

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,TORONTO,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


## Retrieve data into Pandas

In this step we will retrieve the contents of the INSTRUCTOR table into a Pandas dataframe

In [12]:
import ibm_db_dbi

In [13]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [14]:
#retrieve the query results into a pandas dataframe
pdf = pd.read_sql(select_query,pconn)

In [15]:
#print the entire data frame
pdf

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,TORONTO,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


In [16]:
pdf.shape

(3, 5)

## Close the Connection

In [17]:
ibm_db.close(conn)

True